In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
#Import the Player File
Projections = pd.read_csv('DKProjectionsJuly12.csv')

In [3]:
Projections.drop(columns='Unnamed: 0', inplace=True)

In [4]:
Players = Projections.as_matrix()
n, d = Players.shape
n

124

In [5]:
#Split the Players.
SplitPlayers = []
for i in range(n):
    Pos = Players[i,2].split('/')
    if len(Pos) > 1:
        Copy = Players[i].copy()
        Players[i,2] = Pos[0]
        Copy[2] = Pos[1]
        Copy[0] = Copy[0] + ' ' + Pos[1]
        SplitPlayers.append(Copy)

In [6]:
Players

array([['Max Scherzer', 'Nationals', 'SP', 13300, 31.44, 30.18,
        141.13838021905013],
       ['James Paxton', 'Mariners', 'SP', 12500, 15.96, 19.77,
        137.02249884379688],
       ['Blake Snell', 'Rays', 'SP', 11700, 15.48, 17.28,
        117.78698507414148],
       ['Luis Severino', 'Yankees', 'SP', 10800, 18.73, 20.79,
        141.7986483887356],
       ['Corey Kluber', 'Indians', 'SP', 10200, 23.49, 23.39,
        160.2558300566215],
       ['Ross Stripling', 'Dodgers', 'RP', 9100, 21.41, 21.85,
        74.50591066481994],
       ['Tyler Skaggs', 'Angels', 'SP', 7700, 15.68, 16.89,
        97.77655892287477],
       ['J.A. Happ', 'Blue Jays', 'SP', 7400, 13.06, 16.33,
        105.3359263558502],
       ['David Price', 'Red Sox', 'SP', 7100, 15.82, 19.71,
        129.46061280639557],
       ['Kyle Gibson', 'Twins', 'SP', 6700, 12.63, 14.33,
        108.91064554102259],
       ['Mookie Betts', 'Red Sox', 'OF', 6200, 10.91, 9.97,
        98.48591311921871],
       ['Mike Tr

In [7]:
SplitPlayers = np.array(SplitPlayers)

In [8]:
SplitPlayers

array([['Max Muncy 3B', 'Dodgers', '3B', 4900, 10.35, 8.9,
        31.03105122511197],
       ['Eric Thames OF', 'Brewers', 'OF', 4700, 8.78, 9.11,
        51.76881566235962],
       ['Cody Bellinger OF', 'Dodgers', 'OF', 4600, 10.13, 9.02,
        80.5914256198347],
       ['Steve Pearce OF', 'Red Sox', 'OF', 4500, 8.25, 7.83,
        34.35919241917133],
       ['Chris Taylor SS', 'Dodgers', 'SS', 4300, 7.65, 6.62,
        43.62849953076597],
       ['Daniel Robertson SS', 'Rays', 'SS', 4000, 8.03, 6.24,
        32.03245563271605],
       ['Eduardo Nunez SS', 'Red Sox', 'SS', 3800, 7.61, 6.49,
        43.14582791363164],
       ['Brock Holt SS', 'Red Sox', 'SS', 3700, 6.41, 6.1,
        33.579343262796094],
       ['Dee Gordon OF', 'Mariners', 'OF', 3600, 7.27, 7.17,
        65.42195311702889],
       ['David Fletcher 3B', 'Angels', '3B', 3600, 6.8, 6.48,
        18.367346938775512],
       ['Brad Miller 2B', 'Brewers', '2B', 3600, 7.14, 5.85,
        39.147005113221326],
       ['Jef

In [9]:
TotalPlayers = np.concatenate([Players,SplitPlayers])
TotalPlayers

array([['Max Scherzer', 'Nationals', 'SP', 13300, 31.44, 30.18,
        141.13838021905013],
       ['James Paxton', 'Mariners', 'SP', 12500, 15.96, 19.77,
        137.02249884379688],
       ['Blake Snell', 'Rays', 'SP', 11700, 15.48, 17.28,
        117.78698507414148],
       ['Luis Severino', 'Yankees', 'SP', 10800, 18.73, 20.79,
        141.7986483887356],
       ['Corey Kluber', 'Indians', 'SP', 10200, 23.49, 23.39,
        160.2558300566215],
       ['Ross Stripling', 'Dodgers', 'RP', 9100, 21.41, 21.85,
        74.50591066481994],
       ['Tyler Skaggs', 'Angels', 'SP', 7700, 15.68, 16.89,
        97.77655892287477],
       ['J.A. Happ', 'Blue Jays', 'SP', 7400, 13.06, 16.33,
        105.3359263558502],
       ['David Price', 'Red Sox', 'SP', 7100, 15.82, 19.71,
        129.46061280639557],
       ['Kyle Gibson', 'Twins', 'SP', 6700, 12.63, 14.33,
        108.91064554102259],
       ['Mookie Betts', 'Red Sox', 'OF', 6200, 10.91, 9.97,
        98.48591311921871],
       ['Mike Tr

In [10]:
#Create the model
m = Model('Binary Quadratic Model')

Academic license - for non-commercial use only


In [11]:
#Assign the Varplayers and copys -- Returns Dictonary
varPlayers = m.addVars(TotalPlayers[:,0], vtype=GRB.BINARY)

In [12]:
#Put in position constraints.
for name in SplitPlayers[:,0]:
    sNames = name.split(' ')
    #print(name)
    
    #Get the matching platers.
    player1 = varPlayers[name]
    player2 = varPlayers[sNames[0]+' '+sNames[1]]
    
    #Add Constraint
    m.addConstr(player1 + player2 <=1,sNames[0]+' '+sNames[1] +' Postion C' )
m.update()

In [13]:
Salaries = np.array([x[3]*varPlayers[x[0]] for x in TotalPlayers])
m.addConstr(np.sum(Salaries)<=50000, 'Salary C')
m.update()

In [14]:
x = TotalPlayers[3]
x[0]

'Luis Severino'

In [15]:
#Build NP Arrays for each position
OFList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='OF'])
SSList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='SS'])
TBList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='3B'])
SBList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='2B'])
FBList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='1B'])
CList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='C'])
SPList = np.array([varPlayers[x[0]] for x in TotalPlayers if x[2]=='SP'])

In [16]:
#Add Position Constraints.
m.addConstr(np.sum(OFList) ==3, 'OF Constraint')
m.addConstr(np.sum(SSList) ==1, 'SS Constraint')
m.addConstr(np.sum(TBList) ==1, '3B Constraint')
m.addConstr(np.sum(SBList) ==1, '2B Constraint')
m.addConstr(np.sum(FBList) ==1, '1B Constraint')
m.addConstr(np.sum(CList) ==1, 'C Constraint')
m.addConstr(np.sum(SPList) ==2, 'SP Constraint')
m.update()

In [17]:
#Add Team Constraints
Teams = np.unique(TotalPlayers[:,1])
Teams

array(['Angels', 'Blue Jays', 'Brewers', 'Dodgers', 'Indians', 'Mariners',
       'Mets', 'Nationals', 'Padres', 'Pirates', 'Rays', 'Red Sox',
       'Twins', 'Yankees'], dtype=object)

In [18]:
for team in Teams:
    TeamMates = np.array([varPlayers[x[0]] for x in TotalPlayers if x[1]==team and x[2] != 'SP'])
    m.addConstr(np.sum(TeamMates)<=5, team+' Hitter C')
m.update()

In [19]:
#Construct sparse covariance Matrix.
nPlayers, dplayers = TotalPlayers.shape
nPlayers

142

In [20]:
Cov = np.eye(nPlayers)

In [21]:
#Set up the Objective function.
Lamb = 1
objmu  = np.sum(np.array([x[4]*varPlayers[x[0]] for x in TotalPlayers]))
objCov = np.dot(m.getVars(),np.dot(np.array(m.getVars()),Cov))

In [22]:
#Solve.
obj = objmu + Lamb*objCov

In [25]:
Players = np.array([varPlayers[x[0]] for x in TotalPlayers])

In [27]:
np.dot(Players,np.dot(Players,Cov))

<gurobi.QuadExpr: 0.0 + [ C0 ^ 2 + 0.0 C0 * C1 + 0.0 C0 * C2 + 0.0 C0 * C3 + 0.0 C0 * C4 + 0.0 C0 * C5 + 0.0 C0 * C6 + 0.0 C0 * C7 + 0.0 C0 * C8 + 0.0 C0 * C9 + 0.0 C0 * C10 + 0.0 C0 * C11 + 0.0 C0 * C12 + 0.0 C0 * C13 + 0.0 C0 * C14 + 0.0 C0 * C15 + 0.0 C0 * C16 + 0.0 C0 * C17 + 0.0 C0 * C18 + 0.0 C0 * C19 + 0.0 C0 * C20 + 0.0 C0 * C21 + 0.0 C0 * C22 + 0.0 C0 * C23 + 0.0 C0 * C24 + 0.0 C0 * C25 + 0.0 C0 * C26 + 0.0 C0 * C27 + 0.0 C0 * C28 + 0.0 C0 * C29 + 0.0 C0 * C30 + 0.0 C0 * C31 + 0.0 C0 * C32 + 0.0 C0 * C33 + 0.0 C0 * C34 + 0.0 C0 * C35 + 0.0 C0 * C36 + 0.0 C0 * C37 + 0.0 C0 * C38 + 0.0 C0 * C39 + 0.0 C0 * C40 + 0.0 C0 * C41 + 0.0 C0 * C42 + 0.0 C0 * C43 + 0.0 C0 * C44 + 0.0 C0 * C45 + 0.0 C0 * C46 + 0.0 C0 * C47 + 0.0 C0 * C48 + 0.0 C0 * C49 + 0.0 C0 * C50 + 0.0 C0 * C51 + 0.0 C0 * C52 + 0.0 C0 * C53 + 0.0 C0 * C54 + 0.0 C0 * C55 + 0.0 C0 * C56 + 0.0 C0 * C57 + 0.0 C0 * C58 + 0.0 C0 * C59 + 0.0 C0 * C60 + 0.0 C0 * C61 + 0.0 C0 * C62 + 0.0 C0 * C63 + 0.0 C0 * C64 + 0.0 C0 * C65 +